In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from pathlib import Path
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

In [4]:
# Path for submission_files
path = Path("submission_files")
# sub_files = sorted(os.listdir(path / 'submission_files'))

# Import submission example
submission = pd.read_csv("sample_submission.csv", index_col="id")

# Import labels
labels = pd.read_csv("train_labels.csv", index_col="id")

# Create indexes
submission_idx = submission.index
labels_idx = labels.index

In [5]:
df = pd.DataFrame([])

N_DATASETS = 5000

# Grab a number of files
file_list = []

for file in sorted(os.listdir(path)[:N_DATASETS]):
    file_list.append(file)
    temp = pd.read_csv(f"{str(path)}/{file}", usecols= [1])
    df = pd.concat([df,temp], axis= 1, join= "outer")

In [6]:
X = df
X = X.loc[:, X.max() <= 1]
X = X.loc[:, X.min() >= 0]
X.shape

(40000, 4892)

In [150]:
N = 3000
X["id"] = X.index + 1
train_x = X.drop('id', axis = 'columns')
new_train_x = train_x.iloc[: , :N]
train_y = X.pop('id')
new_train_x

,pred,pred,pred,pred,pred,pred,pred,pred,pred,pred,...,pred,pred,pred,pred,pred,pred,pred,pred,pred,pred
0,0.709336,0.799007,0.851891,0.537158,0.623930,0.705970,0.503437,0.633185,0.641550,0.666604,...,0.70329,0.814529,0.804573,0.743812,0.761764,0.776229,0.712757,0.772366,0.685580,0.756885
1,0.452988,0.364453,0.567582,0.354468,0.513818,0.584119,0.454809,0.238501,0.472171,0.522314,...,0.40461,0.575175,0.570035,0.732240,0.573857,0.579728,0.603431,0.606986,0.593099,0.585475
2,0.675462,0.842260,0.800013,0.525229,0.692071,0.715418,0.651008,0.609124,0.691198,0.609994,...,0.92087,0.812807,0.827220,0.814721,0.870502,0.847800,0.838718,0.829428,0.833101,0.818774
3,0.481046,0.577118,0.683032,0.541356,0.630088,0.664514,0.413373,0.508210,0.526140,0.584565,...,0.28333,0.741290,0.784980,0.752206,0.755717,0.790050,0.710334,0.754646,0.714927,0.712135
4,0.957339,0.910337,0.917322,0.874487,0.787595,0.854273,0.843846,0.876749,0.821128,0.913054,...,0.92904,0.938055,0.950617,0.949661,0.942820,0.944026,0.930375,0.942574,0.940509,0.939374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.382515,0.288985,0.483297,0.281330,0.414223,0.463336,0.390367,0.229408,0.455481,0.323629,...,0.12012,0.501956,0.495593,0.504020,0.501509,0.493032,0.481181,0.492261,0.507795,0.499180
39996,0.352498,0.300423,0.594827,0.345519,0.488960,0.506300,0.216482,0.291425,0.471500,0.315838,...,0.64058,0.573505,0.572751,0.587649,0.517492,0.575828,0.579748,0.607065,0.605333,0.570415
39997,0.577554,0.673746,0.760426,0.483366,0.682265,0.663974,0.448841,0.537298,0.575501,0.606992,...,0.72669,0.665126,0.699156,0.723654,0.755989,0.751368,0.801865,0.797185,0.753423,0.726279
39998,0.712353,0.791621,0.807093,0.809911,0.654159,0.667119,0.497323,0.950074,0.520980,0.545782,...,0.75802,0.809951,0.815181,0.874990,0.768542,0.847043,0.715426,0.814039,0.793590,0.775797


In [151]:
len(new_train_x.keys())

3000

In [152]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation = 'relu', input_shape = [len(new_train_x.keys())]),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(16, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001)
    
    model.compile(loss = 'mae', optimizer = optimizer, metrics = ['mae', 'mse'])
    
    return model

In [153]:
model = build_model()

In [154]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 64)                192064    
                                                                 
 dense_49 (Dense)            (None, 32)                2080      
                                                                 
 dense_50 (Dense)            (None, 16)                528       
                                                                 
 dense_51 (Dense)            (None, 1)                 17        
                                                                 
Total params: 194,689
Trainable params: 194,689
Non-trainable params: 0
_________________________________________________________________


In [155]:
prediction = model.predict(new_train_x)

1250/1250 [==============================] - 1s 1ms/step


In [156]:
pred_df = pd.DataFrame(prediction)
pred_df.columns = ['pred']
new_prediction = pred_df.drop(pred_df.index[:20000])
#new_prediction_one = new_prediction[new_prediction['pred'] >= 1]
new_prediction

,pred
20000,0.552343
20001,0.551373
20002,0.527746
20003,0.535617
20004,0.524529
...,...
39995,0.526803
39996,0.525319
39997,0.555510
39998,0.550015


In [157]:
pred_df_result = np.clip(new_prediction, 0.025, 0.975)
np.mean(pred_df_result["pred"])

0.5441808700561523

In [158]:
submission["pred"] = pred_df_result
submission.head()

,pred
id,
20000,0.552343
20001,0.551373
20002,0.527746
20003,0.535617
20004,0.524529


In [159]:
submission.to_csv('submission.csv')